In [1]:
#module file containing all bits needed in python
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import SyncRNG_Python as syncrng
import pandas as pd

import os
from scipy.stats import norm

In [2]:
plt.close()
def plot_matrix(mat, save=False, name=None):
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    mpl.rc('image', cmap='cool')
    plt.figure()
    plt.imshow(mat)
    plt.colorbar()
    if save:
        plt.savefig(name)
    plt.close()

def save_matrix(mat, name):
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import numpy as np

    np.savetxt(name,mat, delimiter=',')

def make_mask(mat, contemp):
    size = np.shape(mat)[0]
    mask = np.ones((size,size))

    if contemp:
        np.fill_diagonal(mask,0)
    
    inds = np.where(mat==0)
    mask[inds]=0
    
    return mask

def coeff_draw_from_cov(amplitude, mat, cov, mask):
    size = len(cov)
    random_matrix = syncrng.generate_random_matrix(size, mu=0, sigma=1, rng=syncrng.universal_rng)
    print(random_matrix)
    sample_step = np.matmul(cov, np.dot(amplitude, random_matrix)) 
    print(sample_step)
    stepmat = mat + sample_step  
    noisy_mat = np.multiply(stepmat, mask)  

    return noisy_mat


def generate_timeseries(start, length, contempamp, contempmat, contempcov, lagamp, lagmat, lagcov, measurecov, save=False):
    ''' Using Y = Ylag*lagmat + Y*contempmat
    so Y= (Ylag*lagmat)*(I-contempmat)^-1
    '''
    size = np.shape(start)[0]
    samples = np.zeros((length + 100, size))
    samples[0, :] = start

    lagmask = make_mask(lagmat, contemp=False)
    contempmask = make_mask(contempmat, contemp=True)

    noisy_lagmat = coeff_draw_from_cov(lagamp, lagmat, lagcov, lagmask)
    noisy_contempmat = coeff_draw_from_cov(contempamp, contempmat, contempcov, contempmask)

    if save:
        save_matrix(noisy_lagmat, 'noisy_lagmat.csv')
        plot_matrix(noisy_lagmat, True, 'noisy_lagmat.png')
        save_matrix(noisy_contempmat, 'noisy_contempmat.csv')
        plot_matrix(noisy_contempmat, True, 'noisy_contempmat.png')

    for i in range(1, length + 100):
        samples[i, :] = (np.matmul(np.matmul(samples[i - 1, :], noisy_lagmat), np.linalg.inv(np.eye(size) - noisy_contempmat)) + start)
        samples[i, :] += syncrng.generate_mvn_samples(mean_vector=np.zeros(size),cov_matrix=measurecov,rng=syncrng.universal_rng)
              
    samples -= start
    samples = samples[101:, :]
    return samples

def clip_timeseries(timeseries, indices_to_clip, min_vec, max_vec):

    ts = timeseries.copy()

    for i in indices_to_clip:
        min_val = min_vec[i]
        max_val = max_vec[i]
        min_inds = np.where(timeseries[:, i] < min_val)[0]
        ts[min_inds, i] = min_val
        max_inds = np.where(timeseries[:, i] > max_val)[0]
        ts[max_inds, i] = max_val

    return ts

    
def clip_outliers(timeseries, sigma, measure_amp, debug, start, contempamp, contempmat, contempcov, lagamp, lagmat, lagcov, measurecov):
    ts = timeseries.copy()
    size = np.shape(timeseries)[1]

    for i in range(size):
        min_val = -sigma * np.sqrt(measure_amp)
        max_val = sigma * np.sqrt(measure_amp)

        # Process indices where values are below the minimum
        min_inds = np.where(timeseries[:, i] < min_val)[0]
        for minind in min_inds:
            count = 0
            tmp = generate_timeseries(ts[minind - 1, :], 3, contempamp, contempmat, contempcov, lagamp, lagmat, lagcov, measurecov, save=False)[1]
            while ts[minind, i] < min_val and count < 2:
                tmp = generate_timeseries(ts[minind - 1, :], 3, contempamp, contempmat, contempcov, lagamp, lagmat, lagcov, measurecov, save=False)[1]
                ts[minind, i] = tmp[i]
                count += 1
                if count == 1: 
                    ts[minind, i] = min_val * (1 - 0.01 * syncrng.universal_rng.rand())

        # Process indices where values exceed the maximum
        max_inds = np.where(timeseries[:, i] > max_val)[0]
        for maxind in max_inds:
            count = 0
            tmp = generate_timeseries(ts[maxind - 1, :], 3, contempamp, contempmat, contempcov, lagamp, lagmat, lagcov, measurecov, save=False)[1]
            while ts[maxind, i] > max_val and count < 2:
                tmp = generate_timeseries(ts[maxind - 1, :], 3, contempamp, contempmat, contempcov, lagamp, lagmat, lagcov, measurecov, save=False)[1]
                ts[maxind, i] = tmp[i]
                count += 1
                if count == 1: 
                    ts[maxind, i] = max_val * (1 - 0.01 * syncrng.universal_rng.rand())

    return ts

In [3]:
# TEST
data = pd.read_csv("~/Desktop/CIFAR/220429.FinalMatrices/1.csv").values
size = 6
start = np.zeros(size)
steps = 3
start_index = size
stop = data.shape[1]

# Extract matrices
matContemp = data[:, start_index:stop]
matLagged = data[:, :size]
print(matContemp)
print(matLagged)

# Generate random covariance matrices
covContemp = syncrng.generate_random_matrix(size, mu=0, sigma=1)
covLagged = syncrng.generate_random_matrix(size, mu=0, sigma=1)
print(covContemp)
print(covLagged)

# Set amplitudes and measurement covariance
ampContemp = 0.01
ampLagged = 0.01
ampMeasure = 1.0
measureCov = ampMeasure * np.eye(size)

# Generate masks
maskContemp = make_mask(matContemp, contemp=True)
maskLagged = make_mask(matLagged, contemp=False)

# Clipping parameters
clip_indices = [0, 1]
clip_mins = [0.5, 0.7]
clip_maxs = [0.8, 1.3]
clip_sigma = 3  # Smaller clip_sigma --> more clipping

# Generate the time series
ts = generate_timeseries(
    start=start,
    length=steps,
    contempamp=ampContemp,
    contempmat=matContemp,
    contempcov=covContemp,
    lagamp=ampLagged,
    lagmat=matLagged,
    lagcov=covLagged,
    measurecov=measureCov,
    save=False
)

# Clip the time series
ts_clipped = clip_outliers(
    timeseries=ts,
    sigma=clip_sigma,
    measure_amp=ampMeasure,
    debug=False,
    start=start,
    contempamp=ampContemp,
    contempmat=matContemp,
    contempcov=covContemp,
    lagamp=ampLagged,
    lagmat=matLagged,
    lagcov=covLagged,
    measurecov=measureCov
)

[[0.         0.         0.         0.         0.         0.25154147]
 [0.         0.         0.33211322 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.32217435 0.21944228 0.         0.         0.        ]
 [0.         0.         0.         0.25807574 0.         0.        ]]
[[ 0.          0.          0.          0.          0.         -0.33930269]
 [ 0.          0.62445427  0.40214014  0.          0.          0.33351903]
 [ 0.2732563   0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.36666885 -0.40415496  0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.44462609]]
[[ 2.86876349e+00  1.37761687e-01  7.44037101e-03  2.51816813e-01
   4.17889602e+00  2.64606077e-02]
 [ 5.24484577e-01  5.63038687e-02

In [ ]:
ts

In [ ]:
# Plot original vs clipped timeseries for parameter 1
days = np.arange(1, steps)
df = pd.DataFrame({
    "days": days,
    "original": ts[:, 0],  # Only param 1
    "clipped": ts_clipped[:, 0]
})

plt.figure(figsize=(10, 6))
plt.plot(df["days"], df["original"], label="Original", linestyle="solid", color="blue")
plt.plot(df["days"], df["clipped"], label="Clipped", linestyle="dashed", color="red")
plt.xlabel("time [days]")
plt.ylabel("value")
plt.title("Original vs. Clipped Timeseries")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Test for consistency
results = []
for _ in range(3):  # Run the function multiple times
    ts = generate_timeseries(
        start=start,
        length=steps,
        contempamp=ampContemp,
        contempmat=matContemp,
        contempcov=covContemp,
        lagamp=ampLagged,
        lagmat=matLagged,
        lagcov=covLagged,
        measurecov=measureCov,
        save=False
    )
    results.append(ts)
    print(ts)

# Check if all generated time series are identical
consistent = all(np.array_equal(results[0], res) for res in results[1:])
print("Is the output consistent across runs?", consistent)


In [ ]:
for i in range(3): 
    print(syncrng.generate_random_matrix(size, mu=0, sigma=1))

In [ ]:
np.dot(3, np.array([1,2,3]))